In [1]:
import os
import numpy as np
import subprocess as sp
import pickle as pkl
import pandas as pd
import shutil

In [2]:
mod_dir = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys'

In [3]:
hgs_mod_dir = os.path.join(mod_dir,'hgs')
model_name = 'lys'
grok_file_path = os.path.join(hgs_mod_dir,model_name + '_e.grok')

In [4]:
grok_file_stem = model_name + '_e'

In [5]:
e_mapping_pkl_path = r'C:\\Users\\southa0000\\Documents\\HGS-DSSAT\\HGS-DSSAT\\examples\\lys\\mapping\\lys_elem_mapping.p'

In [6]:
def CreateCoupledModelDir(mod_dir):
    # Create directories if they don't exist
    coupled_mod_dir = os.path.join(mod_dir,'coupled')
    try:
        os.mkdir(coupled_mod_dir)
    except:
        print(coupled_mod_dir + ' already exists')
    # Create hgs subdirectory
    coupled_mod_hgs_dir = os.path.join(coupled_mod_dir,'hgs')
    try:
        os.mkdir(coupled_mod_hgs_dir)
    except:
        print(coupled_mod_hgs_dir + ' already exists')
    # Create dssat subdirectory
    coupled_mod_dssat_dir = os.path.join(coupled_mod_dir,'dssat')
    try:
        os.mkdir(coupled_mod_dssat_dir)
    except:
        print(coupled_mod_dssat_dir + ' already exists')
    return coupled_mod_dir,coupled_mod_hgs_dir,coupled_mod_dssat_dir

In [7]:
# Create Directory Structure
cmod_dir,cmod_hgs_dir,cmod_dssat_dir = CreateCoupledModelDir(mod_dir)

C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\dssat already exists


In [ ]:
def BuildDailyFluxFiles(e_mapping_pkl_path,coupled_mod_dssat_dir,coupled_mod_hgs_dir,grok_file_stem,day):
    # # Get pm file path
    # pm_file = grok_file_stem.split('_')[0] + '_eday{}o.pm.dat'.format(day-1)
    pm_file = grok_file_stem.split('_')[0] + '_eo.pm.dat'
    pm_file_path = os.path.join(coupled_mod_hgs_dir,pm_file)
    # Load mapping pickle
    with open(e_mapping_pkl_path,'rb') as file:
        map_dict = pkl.load(file)
    # Load lines
    with open(pm_file_path,'r') as file:
        lines = file.readlines()
    # # Get last entry in file for solution time = 1
    # start = [i for i, line in enumerate(lines) if 'SOLUTIONTIME=1.00000000000000' in line][0]
    # Get entry in file for solution time = day
    start = [i for i, line in enumerate(lines) if 'SOLUTIONTIME={}.'.format(day) in line][0]
    # Get indexes for start and end of z flux section
    st1lines = lines[start:]
    start_z = st1lines.index('# z flux (cell centred)\n')
    sub_lines = st1lines[start_z:]
    inds = [i for i, line in enumerate(sub_lines) if '#' in line]
    zlines = sub_lines[inds[0]+1:inds[1]]
    hgs_z_vals = []
    for line in zlines:
        for num in line.strip().split():
            hgs_z_vals.append(float(num))
    # Get lists of dssat models and layers
    dssat_models = set([map_dict[x][0] for x in map_dict.keys()])
    layers = set([map_dict[x][1] for x in map_dict.keys()])
    # For each dssat model, grab a list of elements for each layer
    els_dict = {}
    for mdl in dssat_models:
        els_dict[mdl] = []
        for layer in layers:
            for el in map_dict.keys():
                if map_dict[el] == [mdl,layer]:
                    els_dict[mdl].append(el)
                    break
        # Construct z val list
        z_vals = []
        for el in els_dict[mdl]:
            z_vals.append(-1*hgs_z_vals[-1*el]*100.)
        # Write out to file
        drn_path = os.path.join(coupled_mod_dssat_dir,'{}_{}_DRN.inp'.format(mdl,day))
        with open(drn_path,'w') as file:
            line = ''
            for val in z_vals:
                line += (f"{val:7.4f}"+'   ')
            for i in np.arange(0,10):
                line += ("0.0000"+'   ')
            file.write(line)

In [13]:
# Create Flux Files
for day in np.arange(1,276):
    BuildDailyFluxFiles(e_mapping_pkl_path,cmod_dssat_dir,cmod_hgs_dir,grok_file_stem,day)